# Data access and discovery

## Authors & Contributors

### Authors

- Pier Lorenzo Marasco, Ispra (Italy), [@pl-marasco](https://github.com/pl-marasco)
- Alejandro Coca-Castro, The Alan Turing Institute (United Kingdom), [@acocac](https://github.com/acocac)
- Anne Fouilloux, Simula Research Laboratory (Norway), [@annefou](https://github.com/annefou)

### Contributors
- Tina Odaka, Ifremer (France), [@tinaok](https://github.com/tinaok)

<div class="alert alert-info">
    <i class="fa-question-circle fa" style="font-size: 22px;color:#666;"></i> <b>Overview</b>
    <br>
    <br>
    <b>Questions</b>
    <ul>
        <li>How to access online (remote) datasets?</li>
        <li>How to prepare and discover online geoscience datasets?</li>
        <li>What is Analysis Ready, Cloud Optimized data (ARCO)?</li>
        <li>What is pangeo-forge?</li>
        <li>What is stac?</li>
    </ul>
    <b>Objectives</b>
    <ul>
        <li>Learn to access datasets from online object storage</li>
        <li>Learn about preparing and discovery online datasets</li>
        <li>Learn about Analysis Cloud Optimized (ARCO) data</li>
        <li>Learn about the Pangeo Forge initiative</li>
        <li>Learn about stac</li>
    </ul>
</div>

## Context

We will be using CMIP6 data and access them through [S3-compatible storage](https://en.wikipedia.org/wiki/Amazon_S3) using [Pangeo Catalog](https://pangeo-data.github.io/pangeo-cmip6-cloud/pangeo_catalog.html). 

We will also discuss the generation of Cloud-Optimised Datasets by introducing the [pangeo-forge](https://pangeo-forge.org) initiative. Finally, we will explore [Sentinel-2 Cloud-Optimised Dataset](https://registry.opendata.aws/sentinel-2-l2a-cogs/) online through SpatioTemporal Asset Catalogs ([STAC](https://stacspec.org/en)).

## Setup

This episode uses the following main Python packages:

- xarray {cite:ps}`c-xarray-hoyer2017` with [`netCDF4`](https://pypi.org/project/h5netcdf/) and [`h5netcdf`](https://pypi.org/project/h5netcdf/) engines
- intake-esm {cite:ps}`c-intake-esm`
- s3fs {cite:ps}`c-s3fs-2016`

Please install these packages if not already available in your Python environment.

### Packages

In this episode, Python packages are imported when we start to use them. However, for best software practices, we recommend you to install and import all the necessary libraries at the top of your Jupyter notebook.

### Introduction to CMIP6 data

Coupled Model Intercomparison project Phase 6
- Project under World Climate Research Programme (WCRP) 
- Since 1995 CMIP has coordinated climate model experiments 
- Defines common experiment protocols, forcings and output. 
- 33 model groups participate


#### Very useful links:
- Database for data request: http://clipc-services.ceda.ac.uk/dreq/index.html
    - **-->SEARCH FOR VARIABLES**: http://clipc-services.ceda.ac.uk/dreq/mipVars.html
    - Search for experiments: http://clipc-services.ceda.ac.uk/dreq/experiments.html
- Overview: https://www.wcrp-climate.org/wgcm-cmip/wgcm-cmip6
- ES-DOCs: https://search.es-doc.org/


#### Other links:
GMD special issue with articles explaining all MIPs in CMIP6 :
https://www.geosci-model-dev.net/special_issue590.html

- General CMIP6 website https://www.wcrp-climate.org/wgcm-cmip/wgcm-cmip6
- Guidance documents: (https://pcmdi.llnl.gov/CMIP6/)
- Emissions/Forcing datasets (https://esgf-node.llnl.gov/projects/input4mips/)
- Participating Modelling Groups (https://rawgit.com/WCRP-CMIP/CMIP6_CVs/master/src/CMIP6_institution_id.html)
- Model and experiment documentation (https://search.es-doc.org/)
- CMIP6 ESMValTool evaluation and analysis results (https://cmip-esmvaltool.dkrz.de/)

- Emission visualising: https://eccad.aeris-data.fr


#### Advantages:
- Homogenized and standardized outputs
- Same variable name
- Same experiments

#### Experiments (DECK)
![image](https://user-images.githubusercontent.com/17406708/139691209-ec237004-637b-4947-bb12-104d78a2fe44.png)



### S3-compatible Object Storage to access online data

Up to now we have downloaded data locally and then opened with Xarray `open_dataset`. When willing to manipulate large amount of data, this approach is not optimal (since it requires a lot of unnecessary local downloads). Sharing data online as Object Storage allows for data sharing and access to much larger amounts of data.

One of the most popular methods to access online remote data is through Amazon Simple Storage Service (S3) and you don't necessarily need to use Amazon services to benefit from S3 object storage. Many other providers offer S3-compatible object storage that can be accessed in a very similar way.

When using S3-compatible object storage, you still need to list all the files you would like to access. With such amount of data, it would be very cumbersome. This is why a catalog is created: this catalog is a text file (`json` format) which describes where and which data to get. It adds additional metadata too. CMIP6 Pangeo online catalog can be loaded using `intake-esm` Python package.


### Introduction to the Pangeo CMIP6 online catalog

In [ ]:
import xarray as xr

xr.set_options(display_style="html")
import intake
import cftime

### Open CMIP6 online catalog

In [ ]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col

### Search corresponding data


In [ ]:
cat = col.search(
    source_id=["CESM2"],
    experiment_id=["historical"],
    table_id=["Amon"],
    variable_id=["tas"],
    member_id=["r1i1p1f1"],
)
cat.df

## Create dictionary from the list of datasets we found
- This step may take several minutes so be patient!

In [ ]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={"use_cftime": True})

In [ ]:
dataset_list = list(dset_dict.keys())
dataset_list

## Open dataset

- Use `xarray` python package to analyze netCDF dataset
- `open_dataset` allows to get all the metadata without loading data into memory. 
- with `xarray`, we only load into memory what is needed.

In [ ]:
dset = dset_dict[dataset_list[0]]
dset

### Get metadata corresponding to near-surface air temperature (tas)

In [ ]:
print(dset["tas"])

In [ ]:
dset.time.values

In [ ]:
dset.sel(lat=60, lon=10.75, method="nearest").sel(time="1850-12-15")["tas"].values

:::{warning}
The same dataset can be available from different locations e.g. ESGF, Zenodo, S3-compatible Object storage, etc.
How do you know if it corresponds to the very same dataset? You cannot know except if the datasets have a persistent identifier such as a Digital Object Identifier.
It is therefore recommended *1)* to be extra careful about where you get your datasets, and *2)* to double check that the content is exactly what you expect (for instance, you can perform basic quality checks).                                                            
:::

### Access remote files from S3-compatible Object Storage

The Pangeo CMIP6 online catalog is very user-friendly. However, the complete ESGF CMIP6 catalog is close to 20 PB and so far only a very tiny amount of cloud-optimised CMIP6 data has been generated (approximately 1PB).
Therefore, you may have to download CMIP6 data or other kind of data for instance when comparing CMIP6 model outputs with observations.

Rather than downloading datasets locally and individually, it is recommended to share what you download to everyone. Additional datasets are being stored and made 
publicly available in OpenStack Object storage (Swift).

In [ ]:
import s3fs
import xarray as xr

In [ ]:
fs = s3fs.S3FileSystem(
    anon=True, client_kwargs={"endpoint_url": "https://object-store.cloud.muni.cz"}
)

:::{tip}
The parameter `anon` is for `anonymous` and is set to `True` because the data we have stored at `https://object-store.cloud.muni.cz` is public
:::

#### List files and folders in existing buckets

Instead of organizing files in various folders, object storage systems store files in a flat organization of containers (called "buckets"). 

In [ ]:
fs.ls("MODIS")

In [ ]:
fs.ls("MODIS/MOD08_M3.A2000183.061.2017276075622.nc")

#### Access remote files from S3-compatible Object Storage

In [ ]:
s3path = "s3://MODIS/MOD08_M3.A2000183.061.2017276075622.nc"

In [ ]:
modis = xr.open_dataset(fs.open(s3path))

In [ ]:
modis

### Rename dimensions and variables 

In [ ]:
modis = modis.rename_dims(
    {
        "YDim:mod08": "lat",
        "XDim:mod08": "lon",
        "Effective_Optical_Depth_Average_Ocean_Micron_Levels:mod08": "levels",
    }
)

In [ ]:
modis = modis.rename_vars(
    {
        "YDim": "lat",
        "XDim": "lon",
        "Effective_Optical_Depth_Average_Ocean_Micron_Levels": "levels",
    }
)

In [ ]:
modis

### Coordinates

In [ ]:
x = modis.lon.squeeze().reset_coords(drop=True)
y = modis.lat.squeeze().reset_coords(drop=True)
z = modis.levels.squeeze().reset_coords(drop=True)

In [ ]:
modis = modis.assign_coords({"lon": x, "lat": y, "levels": z})

In [ ]:
modis

In [ ]:
modis.sel(lat=60, lon=10.75, method="nearest")[
    "Aerosol_Optical_Depth_Land_Ocean_Mean_Mean"
].values

#### Access multiple remote files 

In [ ]:
s3path = "s3://MODIS/*.nc"

In [ ]:
remote_files = fs.glob(s3path)
remote_files

We need to add a time dimension to concatenate data. For this, we define a function that will be called for each remote file (via the `preprocess` parameter of Xarray `open_mfdataset`.)

In [ ]:
from datetime import datetime

In [ ]:
def paths_to_datetimeindex(paths):
    return [
        datetime.strptime(date.split(".A")[-1].split(".")[0], "%Y%j") for date in paths
    ]

Xarray `open_mfdataset` allows opening multiple files at the same time.

In [ ]:
# Iterate through remote_files to create a fileset
fileset = [fs.open(file) for file in remote_files]

When opening remote files, you can also select the variables you wish to analyze.

In [ ]:
# Create variable used for time axis
time_var = xr.Variable("time", paths_to_datetimeindex(remote_files))
time_var

In [ ]:
# Load in and concatenate all individual GeoTIFFs
modis = xr.concat(
    [
        xr.open_mfdataset(
            [i],
        )
        for i in fileset
    ],
    dim=time_var,
)

In [ ]:
modis

### Apply the same pre-processing as earlier

In [ ]:
modis = modis.rename_dims(
    {
        "YDim:mod08": "lat",
        "XDim:mod08": "lon",
        "Effective_Optical_Depth_Average_Ocean_Micron_Levels:mod08": "levels",
    }
)

In [ ]:
modis = modis.rename_vars(
    {
        "YDim": "lat",
        "XDim": "lon",
        "Effective_Optical_Depth_Average_Ocean_Micron_Levels": "levels",
    }
)

In [ ]:
modis = modis.assign_coords({"longitude": x, "latitude": y, "levels": z})

In [ ]:
modis

:::{tip}
If you use one of xarray’s open methods such as xarray.open_dataset to load netCDF files with the default engine, it is recommended to use decode_coords=”all”. This will load the grid mapping variable into coordinates for compatibility with rioxarray. See [rioxarray documentation](https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html#xarray).
:::

## Preparing and discover online datasets

With the plethora of cloud storage, there are many available online datasets. To ease the preparation and discovery of such datasets, we describe emerging community-driven initiatives promoting standards suited to both geospatial and geoscience communities. Most of the material below is adapted from a previous Pangeo 101 training {cite:ps}`galaxy2022-pangeo`.

:::{tip}
While we provide a general intro to some initiatives, we suggest below a list of FOSS4G 2022 talks with very interesting developments to prepare and discover spatio-temporal datasets in the cloud. Enjoy!

- [STAC Best Practices and Tools](https://talks.osgeo.org/foss4g-2022/talk/9RRYZM/), 2022-08-24, 11:00–11:30
- [Early use of FOSS4G in a space start up](https://talks.osgeo.org/foss4g-2022/talk/HG7RLR/), 2022-08-24, 11:30–12:00
- [Exploring Data Interoperability with STAC and the Microsoft Planetary Computer](https://talks.osgeo.org/foss4g-2022/talk/L3KNY8/), 2022-08-24, 12:10–12:15
- [Serving oblique aerial imagery using STAC and Cloud Optimized Geotiffs](https://talks.osgeo.org/foss4g-2022/talk/SQYE9A/), 2022-08-24, 14:45–15:15
- [Pangeo Forge: Crowdsourcing Open Data in the Cloud](https://talks.osgeo.org/foss4g-2022/talk/DABTGG/). 2022-08-26, 10:00-10:30.
:::

### Analysis Ready, cloud optimized data (ARCO)
When analyzing data at scale, the data format used is key. For years, the main data format was netCDF e.g. Network Common Data Form but with the use of cloud computing and interest in Open Science, different formats are often more suitable.

Formats for analyzing data from the cloud are refered to as "Analysis Ready, Cloud Optimized" data formats or in short ARCO. Find further info about ARCO datasets in {cite:ps}`Abernathey2022-arco`.

What is "Analysis Ready"?
* Think in terms of "Datasets" not "data files"
* No need for tedious homogenizing / cleaning setup guides
* Curated and cataloged

What is "Cloud Optimized"?
* Compatible with object storage e.g. access via HTTP
* Supports lazy access and intelligent subsetting
* Integrates with high-level analysis libraries and distributed frameworks

Instead of having a big dataset, ARCO datasets are chunked appropriately for analysis and have rich metadata (See Figure 1).

<img src="https://github.com/galaxyproject/training-material/blob/696dfecd4c88e59b487a7a3557cfedca6ec5754b/topics/climate/images/arco_data.png?raw=true" align="Left" /></a>

*Fig 1. Example of an ARCO dataset. Source: {cite:ps}`galaxy2022-pangeo`.*

### The Pangeo forge initiative

[Pangeo Forge](https://pangeo-forge.org/) is an open source platform for data **Extraction**, **Transformation**, and **Loading** (ETL). The goal of *Pangeo Forge* is to make it easy to extract data from traditional repositories and deposit this data in cloud object storage in an analysis-ready, cloud optimized (ARCO) format {cite:ps}`galaxy2022-pangeo`.

Pangeo Forge is inspired directly by Conda Forge, a community-led collection of recipes for building conda packages.

It is under active development and the Pangeo community hopes it will play a role in democratizing the publication of datasets in ARCO format.

#### How does Pangeo Forge work?

Pangeo Forge defines the concept of a recipe, which specifies the logic for transforming a specific data archive into an ARCO data store.
All contributions to Pangeo Forge must include an executable Python module, named recipe.py or similar, in which the data transformation logic is embedded (Figure 2).
The recipe contributor is expected to use one of a predefined set of template algorithms defined by Pangeo Forge.
Each of these templated algorithms is designed to transform data of a particular source type into a corresponding ARCO format, and requires only that the contributor populate the template with information unique to their specific data transformation, including the location of the source files and the way in which they should be aligned in the resulting ARCO data store {cite:ps}`Abernathey2022-arco`.

The diagram below looks complicated but like for conda forge most of the process is automated.

<img src="https://www.frontiersin.org/files/Articles/782909/fclim-03-782909-HTML/image_m/fclim-03-782909-g002.jpg" align="Left" /></a>

*Fig 2. A recipe in relation to Pangeo Forge architecture. Source: {cite:ps}`Abernathey2022-arco`.*

The next step after preparing the dataset is then to tell the community where and how to access to your transformed dataset.

This is done by creating a catalog.

### Spatio Temporal Asset Catalogs (STAC)

The [STAC](https://stacspec.org/en/) specification is a common language to describe geospatial information, so it can more easily be worked with, indexed, and discovered.

#### Why STAC?
* Each provider has its own catalog and interface (APIs).
* Every time you want to access a new catalog, you need to change your program.
* We have lots of data providers and each with a bespoke interface.
* It is becoming quickly difficult for programmers who need to design a new data connector each time.

#### Features
- STAC catalogs are extremely simple.
- They are composed of three layers:
    - **Catalogs**
        - **Collections**
            - **Items**
- STAC is very popular for Earth Observation satellite imagery.
- For instance it can be used to access Sentinel-2 in AWS (see Figure 3).

<img src="https://github.com/galaxyproject/training-material/blob/696dfecd4c88e59b487a7a3557cfedca6ec5754b/topics/climate/images/sentinel2_AWS.png?raw=true" align="Left" /></a>

*Fig 3. Example of STAC collection of Sentinel-2 images hosted in AWS.
Source: {cite:ps}`galaxy2022-pangeo`.*


#### STAC and Pangeo Forge
- Pangeo-forge supports the creation of analysis-ready cloud optimized (ARCO) data in cloud object storage from "classical" data repositories.
- STAC is used to create catalogs and goes beyond the Pangeo ecosystem.
- Work is ongoing to figure out the best way to expose Pangeo-Forge-generated data assets via STAC catalogs.

:::{tip}
Pangeo members, Scott Henderson (University of Washington) and Tom Augspurger (Microsoft), provided a great workshop in FOSS4G 2021 covering STAC.

Feel free to explore the GitHub repository of the  [here](https://github.com/pangeo-data/foss4g-2021).
:::

<div class="alert alert-success">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Key Points</b>
    <br>
    <ul>
        <li>Access to remote dataset</li>
        <li>ARCO datasets</li>
        <li>Pangeo Forge</li>
        <li>STAC</li>
    </ul>
</div>

## References

```{bibliography}
:style: alpha
:filter: topic % "data-access"
```

## Packages citation

```{bibliography}
:style: alpha
:filter: topic % "access" and topic % "package"
:keyprefix: c-
```